In [1]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
from pandas.tseries.offsets import MonthBegin, MonthEnd
from scipy.stats import linregress
from tqdm import tqdm

In [2]:
# import
# pull QB report Average Days to Pay (filter transtype:  invoice, ++ name column, -- total)
df_trans = pd.read_csv('/kaggle/input/qb-apecco/transactions.CSV',
                       encoding='cp1252')

In [3]:
# clean
df_trans.dropna(subset=['Num', 'Avg Days to Pay'], inplace=True)
df_trans['Name'] = df_trans['Name'].str.split(':').str[0]
df_trans['Date'] = pd.to_datetime(df_trans['Date'])

In [4]:
# plot payment trends with trans data
# save as pdf that corresponds to trend
pdf_up = PdfPages("payment_trends_upward.pdf")
pdf_down = PdfPages("payment_trends_downward.pdf")
pdf_none = PdfPages("payment_trends_none.pdf")

lr = None

for name, group in tqdm(df_trans.groupby('Name'), 'generating plots'):
    x = group['Date']
    y = group['Avg Days to Pay']

    x_num = mdates.date2num(x)

    fig = plt.figure(figsize=(6, 4))
    ax_plot = fig.add_subplot()
    ax_plot.scatter(x, y, alpha=0.6)

    trend_category = "none"

    if len(x.unique()) >= 5:
        lr = linregress(x_num, y)
        if lr.pvalue < 0.05:
            x_fit = np.linspace(x_num.min(), x_num.max(), 100)
            y_fit = lr.intercept + lr.slope * x_fit
            ax_plot.plot(x_fit, y_fit, color='red', linewidth=2)

            if lr.slope > 0:
                trend_category = "up"
            elif lr.slope < 0:
                trend_category = "down"
        else:
            trend_category = "none"

    yr_start = pd.Timestamp(year=group['Date'].min().year, month=1, day=1)
    yr_end = pd.Timestamp(year=group['Date'].max().year, month=12, day=31)
    ax_plot.set_xlim([yr_start - MonthBegin(1), yr_end + MonthEnd(1)])
    
    ax_plot.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax_plot.xaxis.set_major_locator(mdates.YearLocator())
    fig.autofmt_xdate(rotation=45)

    ax_plot.set_xlabel('Invoice Date', fontsize=8)
    ax_plot.set_ylabel('Days to Pay', fontsize=8)
    ax_plot.set_title(f'{name}', fontsize=8)

    if lr is not None:
        ax_plot.text(0.05, 0.95, f'slope={lr.slope:.3f}', transform=ax_plot.transAxes)

    ax_plot.tick_params(axis='x', labelsize=8)
    ax_plot.tick_params(axis='y', labelsize=8)

    if trend_category == "up":
        pdf_up.savefig(fig)
    elif trend_category == "down":
        pdf_down.savefig(fig)
    else:
        pdf_none.savefig(fig)

    plt.close(fig)

pdf_up.close()
pdf_down.close()
pdf_none.close()

generating plots: 100%|██████████| 451/451 [00:32<00:00, 13.98it/s]
